In [9]:
#code to read data from snowflake using FosforML
 
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()
 
table_name = 'SALES_CLEAN'
 
sf_df = my_session.sql("select * from {}".format(table_name)) 

df = sf_df.to_pandas() 
 
print(df.shape) 

df.head(100) 

ModuleNotFoundError: No module named 'fosforml'

In [ ]:
type(sf_df)

In [ ]:
type(df)

In [ ]:
df.describe()

In [ ]:
df.corr(numeric_only=True)

In [7]:
df.info()

NameError: name 'df' is not defined

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data=df.copy()

In [ ]:
data[['MNTH_CODE', 'TRANS_DATE', 'START_DATE', 'SALES_VALUE', 'SALES_UNITS',
       'SALES_VOLUME', 'SALES_PTR_VALUE', 'OC_CODE', 'DISTRIBUTOR_CODE',
       'OUTLET_CODE', 'CITY', 'STATE', 'COUNTY', 'STREET', 'PRODUCT_CODE',
       'CATEGORY', 'SUBCATEGORY', 'BRAND']].head()

In [ ]:
import pandas as pd

In [ ]:
data.head()

In [ ]:
data.drop(['TRANS_DATE', 'START_DATE', 'DISTRIBUTOR_CODE',
       'OUTLET_CODE', 'CITY', 'STATE', 'COUNTY', 'STREET', 'PRODUCT_CODE',
       'CATEGORY', 'SUBCATEGORY', 'BRAND'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
Scaler=MinMaxScaler()

In [8]:
Scaler.fit(data)
X_Train_scaled=Scaler.transform(data)

NameError: name 'Scaler' is not defined

In [ ]:
X_Train_scaled

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Assuming X_Train_scaled is already defined and scaled

# Initialize PCA with the number of components you want
pca = PCA(n_components=2)

# Fit and transform the scaled training data
X_Train_pca = pca.fit_transform(X_Train_scaled)

# Convert the PCA result to a DataFrame
X_Train_pca_df = pd.DataFrame(X_Train_pca, columns=['PC1', 'PC2'])

# Perform K-means clustering
kmeans = KMeans(n_clusters=4)  # Adjust the number of clusters as needed
kmeans.fit(X_Train_pca_df)

# Add the cluster labels to the DataFrame
X_Train_pca_df['Cluster'] = kmeans.labels_

# Plot the clusters
plt.figure(figsize=(10, 7))
plt.scatter(X_Train_pca_df['PC1'], X_Train_pca_df['PC2'], c=X_Train_pca_df['Cluster'], cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA and K-means Clustering')
plt.show()

In [ ]:
from sklearn.cluster import KMeans  
 
wcss_list= []  #Initializing the list for the values of WCSS  
 
#Using for loop for iterations from 1 to 10.  
 
for i in range(1, 11):  
 
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state= 42)  
 
    kmeans.fit(X_Train_pca_df)  
 
    wcss_list.append(kmeans.inertia_)  
 
plt.plot(range(1, 11), wcss_list)  
 
plt.title('The Elbow Method Graph')  
 
plt.xlabel('Number of clusters(k)')  
 
plt.ylabel('wcss_list')  
 
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Assuming X_Train_scaled is already defined and scaled

# Initialize PCA with the number of components you want
pca = PCA(n_components=2)

# Fit and transform the scaled training data
X_Train_pca = pca.fit_transform(X_Train_scaled)

# Convert the PCA result to a DataFrame
X_Train_pca_df = pd.DataFrame(X_Train_pca, columns=['PC1', 'PC2'])

# Perform K-means clustering
kmeans = KMeans(n_clusters=4)  # Adjust the number of clusters as needed
kmeans.fit(X_Train_pca_df)

# Add the cluster labels to the DataFrame
X_Train_pca_df['Cluster'] = kmeans.labels_

# Plot the clusters
plt.figure(figsize=(10, 7))
plt.scatter(X_Train_pca_df['PC1'], X_Train_pca_df['PC2'], c=X_Train_pca_df['Cluster'], cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA and K-means Clustering')
plt.show()

In [ ]:
combined_df = pd.concat([df, X_Train_pca_df], axis=1)

In [ ]:
combined_df.head()

In [ ]:
combined_df['Cluster'].value_counts()

In [ ]:
pip install snowflake-connector-python

In [ ]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [ ]:
combined_df['Cluster'].value_counts()


In [ ]:
combined_df['ClusterID'] = combined_df['Cluster']
 
di = {0: "CLUSTER_0", 1: "CLUSTER_1",2: "CLUSTER_2", 3: "CLUSTER_3"}
combined_df.replace({"ClusterID": di})

In [ ]:
#Code to write back dataframe to Snowflake
 
ins_train_sf=my_session.createDataFrame(
        combined_df.values.tolist(),
        schema=combined_df.columns.tolist())
ins_train_sf.write.mode("overwrite").save_as_table("ASSORTMENT_PLANNING.CPG_BRONZE.CLUSTERED_DATA")